In [1]:
from transformers import TFAutoModel
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [2]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [3]:
import tensorflow as tf

In [4]:
input_ids = tf.keras.layers.Input(shape = (512,), name = 'input_ids', dtype = 'int32')
mask = tf.keras.layers.Input(shape = (512,), name = 'attention_mask', dtype = 'int32')

In [5]:
embeddings = bert.bert(input_ids, attention_mask = mask)[1]

In [6]:
x = tf.keras.layers.Dense(1024, activation = 'relu') (embeddings)
y = tf.keras.layers.Dense(5, activation = 'softmax', name = 'outputs') (x)

In [7]:
model = tf.keras.Model(inputs = [input_ids, mask], outputs = y)

In [8]:
model.layers[2].trainable = False

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5, weight_decay = 1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer = optimizer, loss = loss, metrics = [acc])

In [11]:
element_spec = ({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
                 'attention_mask': tf.TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
                tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [12]:
train_ds =tf.data.Dataset.load('train', element_spec = element_spec)
val_ds =tf.data.Dataset.load('val', element_spec = element_spec)
train_ds.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.config.experimental.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5805918924671316742
xla_global_id: -1
]


[]

In [ ]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 3
)

Epoch 1/3
 305/8778 [>.............................] - ETA: 30:28:31 - loss: 1.1819 - accuracy: 0.5539